In [37]:
import cv2
import json
import os
import numpy as np

def load_and_preprocess_data(img_folder, json_folder, img_width=224, img_height=224):
    images = []
    labels = []

    # List all files in the img_folder assuming the naming convention matches your description
    for img_file in sorted(os.listdir(img_folder)):
        # Extract the number from the image file name, assuming the format "image_1.png"
        img_number = img_file.split('_')[1].split('.')[0]
        
        # Construct the full file paths
        img_path = os.path.join(img_folder, img_file)
        label_path = os.path.join(json_folder, f"label_{img_number}.json")

        print(f"Loading image: {img_path}")
        print(f"Loading label: {label_path}")

        # Load and resize image
        image = cv2.imread(img_path)
        if image is None:
            print(f"Failed to load image at {img_path}")
            continue
        image = cv2.resize(image, (img_width, img_height))
        images.append(image.astype('float32') / 255.0)  # Normalize image data to 0-1

        # Load labels
        if not os.path.exists(label_path):
            print(f"Label file not found at {label_path}")
            continue
        
        with open(label_path, 'r') as file:
            data = json.load(file)
            box_data = []
            for item in data['boxes']:
                if item['label'] == 'ball':
                    # Normalize coordinates
                    x = float(item['x']) / data['width']
                    y = float(item['y']) / data['height']
                    width = float(item['width']) / data['width']
                    height = float(item['height']) / data['height']
                    box_data.append([x * img_width, y * img_height, width * img_width, height * img_height])
            labels.append(box_data)

    return np.array(images), labels

# Usage
img_folder = '../images'
json_folder = '../json_labeling'
images, labels = load_and_preprocess_data(img_folder, json_folder)


Loading image: ../images\image_1.png
Loading label: ../json_labeling\label_1.json
Loading image: ../images\image_10.png
Loading label: ../json_labeling\label_10.json
Loading image: ../images\image_100.png
Loading label: ../json_labeling\label_100.json
Loading image: ../images\image_11.png
Loading label: ../json_labeling\label_11.json
Loading image: ../images\image_12.png
Loading label: ../json_labeling\label_12.json
Loading image: ../images\image_13.png
Loading label: ../json_labeling\label_13.json
Loading image: ../images\image_14.png
Loading label: ../json_labeling\label_14.json
Loading image: ../images\image_15.png
Loading label: ../json_labeling\label_15.json
Loading image: ../images\image_16.png
Loading label: ../json_labeling\label_16.json
Loading image: ../images\image_17.png
Loading label: ../json_labeling\label_17.json
Loading image: ../images\image_18.png
Loading label: ../json_labeling\label_18.json
Loading image: ../images\image_19.png
Loading label: ../json_labeling\label_

In [52]:
import cv2
import numpy as np
#displaying image example with
def display_image_with_boxes(image, boxes, window_name="Image", expand_by=4):
    """
    Displays an image with bounding boxes drawn on it, with each box slightly larger.

    Parameters:
    - image: The image array (numpy array).
    - boxes: A list of bounding boxes, each represented as [center_x, center_y, width, height].
    - window_name: The name of the window in which the image will be displayed.
    - expand_by: Number of pixels to increase the size of each box by (total, not per side).
    """
    # Make a copy of the image to draw on
    display_img = image.copy()

    # Draw each box
    for box in boxes:
        center_x, center_y, width, height = map(int, box)  # Ensure the coordinates are integer
        # Increase the box size by expanding by a specified number of pixels
        top_left = (center_x - (width // 2) - (expand_by // 2), center_y - (height // 2) - (expand_by // 2))
        bottom_right = (center_x + (width // 2) + (expand_by // 2), center_y + (height // 2) + (expand_by // 2))
        cv2.rectangle(display_img, top_left, bottom_right, (0, 255, 0), 2)  # Green box

    # Display the image
    cv2.imshow(window_name, display_img)
    cv2.waitKey(0)  # Wait for a key press to close the window
    cv2.destroyAllWindows()

# Example usage (assuming images and labels are already loaded and preprocessed)
if len(images) > 0 and len(labels) > 0:
    display_image_with_boxes(images[0], labels[0])




In [53]:
from PIL import Image, ImageDraw
import numpy as np

def display_image_with_boxes_pil(image, boxes, expand_by=4):
    """
    Displays an image with bounding boxes drawn on it using PIL, with each box slightly larger.
    Also converts the image to grayscale.

    Parameters:
    - image: The image array (numpy array).
    - boxes: A list of bounding boxes, each represented as [center_x, center_y, width, height].
    - expand_by: Number of pixels to increase the size of each box by (total, not per side).
    """
    # Ensure the image is in uint8 format
    if image.dtype != np.uint8:
        image = (255 * image).astype(np.uint8)

    # Convert numpy array image to PIL Image
    pil_img = Image.fromarray(image)
    pil_img = pil_img.convert('L')  # Convert to grayscale

    # Create a drawing context
    draw = ImageDraw.Draw(pil_img)

    # Draw each box
    for box in boxes:
        center_x, center_y, width, height = map(int, box)
        top_left = (center_x - (width // 2) - (expand_by // 2), center_y - (height // 2) - (expand_by // 2))
        bottom_right = (center_x + (width // 2) + (expand_by // 2), center_y + (height // 2) + (expand_by // 2))
        draw.rectangle([top_left, bottom_right], outline="white", width=2)  # White box on grayscale image

    # Display the image
    pil_img.show()

# Example usage (assuming images and labels are already loaded and preprocessed)
if len(images) > 0 and len(labels) > 0:
    display_image_with_boxes_pil(images[0], labels[0])


In [54]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([
    # Input layer & first convolutional layer
    Conv2D(32, (3, 3), activation='relu', input_shape=(480, 640, 1)),  # Adjusted input shape
    MaxPooling2D(2, 2),
    
    # Additional convolutional layers
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(256, (3, 3), activation='relu'),  # Higher number of filters for more complex features
    MaxPooling2D(2, 2),
    
    # Flatten the output of the convolutional layers
    Flatten(),
    
    # Fully connected layers
    Dense(1024, activation='relu'),  # Increased number of neurons for a more complex model
    Dense(512, activation='relu'),
    
    # Output layer
    Dense(4)  # Outputs x, y, width, height for the bounding box
])

# Model compilation
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

model.summary()


c:\Users\hanso\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 478, 638, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 239, 319, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 237, 317, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 118, 158, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 116, 156, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 58, 78, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 56, 76, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 28, 38, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 272384)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1024)           │   278,922,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 4)              │         2,052 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 279,836,932 (1.04 GB)

 Trainable params: 279,836,932 (1.04 GB)

 Non-trainable params: 0 (0.00 B)

In [57]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Load your dataset
# Assuming `images` and `labels` are loaded and preprocessed arrays
# Example: images = np.array([...]) and labels = np.array([...])

# Splitting the dataset into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(640, 480, 1)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(1024, activation='relu'),
    Dense(512, activation='relu'),
    Dense(4)  # Output layer: x, y, width, height
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=10, validation_data=(x_val, y_val), batch_size=32)


ValueError: Unrecognized data type: x=[[[[0.3882353  0.40392157 0.39607844]
   [0.4        0.40784314 0.4       ]
   [0.40392157 0.40784314 0.39607844]
   ...
   [0.1254902  0.16862746 0.25882354]
   [0.14901961 0.19215687 0.28627452]
   [0.1764706  0.21176471 0.30588236]]

  [[0.37254903 0.38039216 0.38039216]
   [0.39215687 0.4        0.4       ]
   [0.39607844 0.40784314 0.39607844]
   ...
   [0.13333334 0.17254902 0.26666668]
   [0.13725491 0.18431373 0.27450982]
   [0.15294118 0.18431373 0.2784314 ]]

  [[0.40784314 0.42352942 0.40784314]
   [0.41568628 0.42352942 0.4117647 ]
   [0.4117647  0.42352942 0.4117647 ]
   ...
   [0.12941177 0.18039216 0.27058825]
   [0.12941177 0.1764706  0.27450982]
   [0.13725491 0.1764706  0.27058825]]

  ...

  [[0.58431375 0.5764706  0.5686275 ]
   [0.5803922  0.58431375 0.57254905]
   [0.5803922  0.58431375 0.57254905]
   ...
   [0.35686275 0.4        0.40784314]
   [0.35686275 0.4        0.40784314]
   [0.3529412  0.39215687 0.40784314]]

  [[0.5

In [55]:
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
import numpy as np

# Assuming 'images' and 'labels' are loaded and preprocessed
x_train, x_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

# Callbacks
checkpoint = ModelCheckpoint('model_best.h5', save_best_only=True)
tensorboard = TensorBoard(log_dir='./logs')

# Start training
history = model.fit(x_train, y_train,
                    validation_data=(x_val, y_val),
                    epochs=10,
                    batch_size=32,
                    callbacks=[checkpoint, tensorboard])


ValueError: The filepath provided must end in `.keras` (Keras model format). Received: filepath=model_best.h5